In [1]:
pip install pdal

Note: you may need to restart the kernel to use updated packages.


PDAL wants nupmy <= 1.20.0, you may need to conda update it ```conda update numpy=1.20.1```pip install pdal

In [1]:
import numpy as np
np.version.version

'1.20.1'

In [69]:
from API_utils import show_dates, show_files_for_site_date
import os
import numpy as np
import json
import multiprocessing
import time
import glob
import rasterio
from osgeo import gdal
import rasterio as rio
import re
import time

ncores = multiprocessing.cpu_count()
ncores

16

For now we will just do TEAKs oldest Lidar flight

In [3]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'
NEON_path = '/home/jovyan/NEON'
date = '2018-06'

os.makedirs(data_path, exist_ok=True)

#show_dates(site, productcode)


In [12]:
def generate_laz_download_info():
    '''Returns: time of url issueance, list of laz files'''
    t0 = time.time()
    files = show_files_for_site_date(productcode, site, '2018-06')
    laz = []
    for file in files:
        if 'classified_point_cloud_colorized.laz' in file['name']:
            laz.append(file)
    return(t0, laz)
    
    
def refresh_url(f, t0):
    '''If too much time has elapsed since url issued, modifies f to contain new url'''
    if time.time() - t0 < 3550:
        pass
    else:
        files = show_files_for_site_date(productcode, site, '2018-06')
        for file in files:
            if file['name'] == f['name']:
                f['url'] = file['url']

In [70]:
import requests
import hashlib


def download_from_NEON_API(f, data_path):

    attempts = 0 
    while attempts < 4:
        try:
            # get the file 
            response = requests.get(f['url'], stream=True)
            
            # raise an error for bad status codes
            response.raise_for_status()
            
            #check the md5 if it exists
            if f['md5']:
                md5 = hashlib.md5(response.content).hexdigest()
                if md5 == f['md5']:
                    success = True
                    attempts = 4
                else:
                    fmd5 = f['md5']
                    print(f'md5 mismatch on attempt {attempts}')
                    success = False
                    attempts = attempts + 1
            else: 
                success = True
                attempts = 4
        except Exception as e:
            print(f'Warning:\n{e}')
            success = False
            attempts = attempts + 1
    # write the file
    if success:
        fname = os.path.join(data_path, f['name'])
        with open(fname, 'wb') as sink:
            for block in response.iter_content(1024):
                sink.write(block)
        time.sleep(1)   # this should not be nededed!
        with open(fname, 'r') as src:
            print(src.profile)
            
    else:
        raise Exception('failed to download')

In [5]:
import pdal
from string import Template
import subprocess
import time

In [6]:
def make_pipe(f, bbox, out_path, resolution=1):
    tile = '_'.join(f.rpartition('/')[2].split('_')[4:6])
    '''Creates, validates and then returns the pdal pipeline
    
    Arguments:
    bbox       -- Tuple - Bounding box in srs coordintes (default srs is EPSG:3857),
                  in the form: ([xmin, xmax], [ymin, ymax]).
    outpath   -- String - Path where the CHM shall be saved. Must include .tif exstension.
    srs        -- String - EPSG identifier for srs  being used. Defaults to EPSG:3857
                  because that is what ept files tend to use.
    threads    -- Int - Number os threads to be used by the reader.ept. Defaults to 4.
    resolution -- Int or Float - resolution (m) used by writers.gdal
    '''
    
    t = Template('''
    {
        "pipeline": [
            {
            "filename": "${f}",
            "type": "readers.las",
            "tag": "readdata"
            },
            {
            "type":"filters.outlier",
            "method":"radius",
            "radius":1.0,
            "min_k":4
            },
            {
            "type":"filters.optimalneighborhood",
            "min_k":8,
            "max_k": 50
            },
            {
            "type":"filters.covariancefeatures",
            "knn":10,
            "threads": 2,
            "feature_set": "all"
            },
            {
            "filename": "${outpath}/${tile}_Anisotropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Anisotropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_DemantkeVerticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "DemantkeVerticality",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Density.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Density",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Eigenentropy.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Eigenentropy",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Linearity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Linearity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Omnivariance.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Omnivariance",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Planarity.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Planarity",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Scattering.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Scattering",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_EigenvalueSum.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "EigenvalueSum",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_SurfaceVariation.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "SurfaceVariation",
            "bounds": "${bbox}"
            },
            {
            "filename": "${outpath}/${tile}_Verticality.tif",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution":  "${resolution}",
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "Verticality",
            "bounds": "${bbox}"
            }
        ]
    }''')

    pipe = t.substitute(f=f, bbox=bbox, outpath=out_path, tile=tile, resolution=resolution)
    pipeline = pdal.Pipeline(pipe)
    if pipeline.validate():
        return(pipeline, tile)
    else:
        raise Exception('Bad pipeline (sorry to be so ambigous)!')

from dask import delayed, compute
from dask.diagnostics import ProgressBar

def make_covariance_layer_mosaic(laz_list, data_path, extent, t0):
    '''makes tifs of each covariance feature'''
    # make a list
    results = []

    # append delayed computations to list
    for f in laz_list:
        results.append(delayed(make_covariance_layers_tiles)(f, data_path, extent, t0))

    # compute the delayed computations in ||
    with ProgressBar():
        computed = compute(*results)

    # computed holds tyuples of name and reason for failed files, filter out Nones (i.e. non-fails)
    computed = [f for f in computed if f != None]    


def make_covariance_layers_tiles(f, data_path, extent, t0):
    '''TODO: make it verify that the same srs is in use fo extent and bbox'''

    # make sure url is still valid
    refresh_url(f, t0)
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    size = f['size']    
    
    # Download the laz
    download_from_NEON_API(f, data_path)
    
    # find the bounds
    tries = 0
    try:
        bbox = extract_bbox(name)
        bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])
        # if the bounds are at least partially within the extent...
        a = bbox['minx'] <= extent['maxx']
        b = bbox['maxx'] >= extent['minx']
        c = bbox['miny'] <= extent['maxy']
        d = bbox['maxy'] >= extent['miny']
        if a and b and c and d:
            # make and execute the pdal pipeline
            pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
            print(tile)
            count = pipeline.execute()
            # remove the laz file
            os.remove(name)
            
        else:
            # remove the laz file
            # TODO: it is wack that i download the laz just to find out it is not needed
            os.remove(name)
    
    except:
        # keep track of failed files and reason for failure
        if os.path.isfile(name):
            os.remove(name)
            reason = 'processing'
        else:
            reason = 'download'
        
        return((f['name'], reason))
    
    
def extract_bbox(name, tries=1):
    cmd = f'pdal info {name}'
    try:
        reply = subprocess.run(cmd, shell=True, capture_output=True, timeout=15)
        if len(reply.stderr) > 0: print(reply.stderr)
        meta = json.loads(reply.stdout)
        bbox = meta['stats']['bbox']['native']['bbox']
        return(bbox)
    except subprocess.TimeoutExpired:
        if tries < 2:
            bbox = extract_bbox(name, tries=tries+1)
        else:
            return(None)

In [59]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

def make_covariance_layer_mosaic(laz_list, data_path, extent, t0):
    '''makes tifs of each covariance feature.
    Uses delayed '''
    # run the pipline to make the covariance features
    results = []
    for f in laz_list:
        results.append(delayed(make_covariance_layers_tiles)(f, data_path, extent, t0))

    with ProgressBar():
        computed = compute(*results)

    # computed holds tuples of name and reason for failed files, filter out Nones (i.e. non-fails)
    computed = [f for f in computed if f != None]
    
    # make a dict of tiles for each layer
    layers = ['Anisotropy', 'DemantkeVerticality', 'Density',
              'Eigenentropy', 'EigenvalueSum', 'Linearity',
              'Omnivariance', 'Planarity', 'Scattering',
              'SurfaceVariation', 'Verticality']

    eigendict ={}
    for layer in layers:
        eigendict[layer] = [os.path.join(data_path,f) for f in os.listdir(data_path) if layer in f]

    # make sure tifs have pixels aligned to origin, do it in || with dask.
    lazy = []
    for key, val in eigendict.items():
        for f in val:
              lazy.append(delayed(origin_warp_if_needed)(f))

    with ProgressBar():
        _ = compute(*lazy)
    
    # build vrt, make mosaic for each layer, in ||
    also_lazy = []
    for layer, files in eigendict.items():    
          also_lazy.append(delayed(make_mosaic)(layer, files, data_path))

    with ProgressBar():
        _ = compute(*also_lazy)

 

    return(computed)




def origin_warp_if_needed(f):
    if not origin_good_1mresolution(f):
        warp(f)

def origin_good_1mresolution(f):
    '''Checks to make sure the origin is centered on 1m resolution'''
    cmd = f'gdalinfo {f} | grep \'Origin =\''
    result = subprocess.run(cmd, shell=True, capture_output=True)
    if len(result.stderr) > 0: print(result.stderr)
        
    #meta = json.loads(result.stdout)
    x, y = re.search('\(([^)]+)', str(result.stdout)).group(1).split(',')
    x, y = float(x), float(y)
    
    # make sure x and y are whole numbers
    soft_pink_truth = x.is_integer() and y.is_integer()
    
    return(soft_pink_truth)


def warp(f, resolution=1):
    '''Runs gdalwarp -tr -tap on the file, f.
    This will ensure that the tif pixels are aligned to the origin'''
    # Note: this will fail if there are more than one . in the fname
    base = f.split('.')[0]
    
    # warp the pixels to ensure they are on origin
    cmd = f'gdalwarp -tr {resolution} {resolution} -tap {f} {base}_w.tiff'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    if len(result.stderr) > 0: print(result.stderr)
    
    # move new file to old file name
    cmd = f'mv {base}_w.tiff {f}'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    if len(result.stderr) > 0: print(result.stderr)

        
def make_mosaic(layer, files, data_path):
    '''makes a mosaic of tiles'''
    # make a vrt for the layer
    vrt = gdal.BuildVRT(os.path.join(data_path, f'{layer}.vrt'), files)

    # make a mosaic for the layer
    mosaic = gdal.Translate(os.path.join(data_path, f'{layer}.tif'), vrt)



def make_covariance_layers_tiles(f, data_path, extent, t0):
    '''TODO: make it verify that the same srs is in use fo extent and bbox'''

    # make sure url is still valid
    refresh_url(f, t0)
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    
    try:
        # Download the laz
        download_from_NEON_API(f, data_path)
    except:
        print('failed to download!')
        reason = 'download'
        return((f['name'], reason))
        
    try:    
        # make and execute the pdal pipeline
        print('Making and executing pipeline:\n')
        pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
        print(tile)
        count = pipeline.execute()
        # remove the laz file
        os.remove(name)

    except:
        # keep track of failed files and reason for failure
        if os.path.isfile(name):
            os.remove(name)
            reason = 'processing'
        else:
            reason = 'download'
        
        return((f['name'], reason))
    

In [73]:
def download_laz(f, target_dir):
    '''Takes an entry from output of generete_laz_download, saves to target_dir'''

    # make sure url is still valid
    refresh_url(f, t0)
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    
    try:
        # Download the laz
        download_from_NEON_API(f, data_path)
        return(None)
    except:
        return(f['name'])

In [ ]:


# make sure dir exists 
os.makedirs(data_path, exist_ok=True)

# get list of files to download
t0, laz = generate_laz_download_info()

# download the files
print('Downlaoding laz files:')
lazy0 = []
for f in laz:
    lazy0.append(delayed(download_laz)(f, data_path))
    
with ProgressBar():
    fails = compute(*lazy0)

fails = [thing for thing in fails if thing!=None]

# if any failed print a message
# nope, for some reason it thinks they all fail
        
# make the pipelines and run them
print('Making and executing pipeline:\n')
        pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
        print(tile)
        count = pipeline.execute()

In [ ]:
# diy glob the files
down_laz = [os.path.join(data_path,f) for f in os.listdir(data_path) if '.laz' in f]

# make the pipelines and run them
print('Making and executing pipeline:')
lazy1 = []
for f in down_laz:
    
def execute_pipe(f)
        
        pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
        count = pipeline.execute()
        
 try:
        bbox = extract_bbox(name)
        bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])
        # if the bounds are at least partially within the extent...
        a = bbox['minx'] <= extent['maxx']
        b = bbox['maxx'] >= extent['minx']
        c = bbox['miny'] <= extent['maxy']
        d = bbox['maxy'] >= extent['miny']
        if a and b and c and d:
            # make and execute the pdal pipeline
            pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
            print(tile)
            count = pipeline.execute()

In [68]:

t0, laz = generate_laz_download_info()

make_covariance_layer_mosaic(laz, data_path, extent, t0)

[                                        ] | 0% Completed |  1.0sfailed to download!
[                                        ] | 0% Completed |  3.6sfailed to download!
[                                        ] | 0% Completed |  7.9sfailed to download!
[                                        ] | 1% Completed |  9.6sfailed to download!
[                                        ] | 1% Completed |  9.8sfailed to download!
[                                        ] | 2% Completed | 12.6sfailed to download!
[                                        ] | 2% Completed | 15.8sfailed to download!
[#                                       ] | 2% Completed | 16.1sfailed to download!
[#                                       ] | 3% Completed | 19.3sfailed to download!
[#                                       ] | 3% Completed | 23.5sfailed to download!
[#                                       ] | 4% Completed | 24.3sfailed to download!
[#                                       ] | 4% Completed | 24.4s

ValueError: Received a NULL pointer.

In [8]:
img_path = os.path.join(NEON_path, 'D17_CHM_all_Mask5m_roughFlightline.tif')

with rio.open(img_path, 'r') as src:
    profile = src.profile
    
profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': 128.0, 'width': 1010, 'height': 10685, 'count': 1, 'crs': CRS.from_epsg(32611), 'transform': Affine(1.0, 0.0, 319344.0,
       0.0, -1.0, 4101685.0), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

In [9]:
from gdal import GA_ReadOnly

img = gdal.Open(img_path, GA_ReadOnly)


width = img.RasterXSize
height = img.RasterYSize
gt = img.GetGeoTransform()
extent = {}
extent['minx'] = gt[0]
extent['miny'] = gt[3] + width*gt[4] + height*gt[5] 
extent['maxx'] = gt[0] + width*gt[1] + height*gt[2]
extent['maxy'] = gt[3]
img = None


t0, laz = generate_laz_download_info()

for f in laz[0:5]:
    make_covariance_layers_tiles(f, data_path, extent, t0)

To run them all

In [17]:
stacks = [os.path.join(data_path, f) for f in os.listdir(data_path) if 'stack' in f]
not_stax = [f for f in os.listdir(data_path) if 'stack' not in f]

for f in not_stax:
    try:
        os.remove(os.path.join(data_path, f))
    except:
        pass
    
del(not_stax)

In [18]:
@delayed
def warp(f):    
    base = f.split('.')[0]
    
    # warp the pixels to ensure they are on origin
    cmd = f'gdalwarp -tr 0.6 0.6 -tap {f} {base}_w.tiff'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    if len(result.stderr) > 0: print(result.stderr)
    
    # move new file to old file name
    cmd = f'mv {base}_w.tiff {f}'
    result = subprocess.run(cmd, shell=True, capture_output=True)
    if len(result.stderr) > 0: print(result.stderr)
        

lazy = []
for f in stacks:
    lazy.append(warp(f))
    
with ProgressBar():
    _ = compute(*lazy)

[########################################] | 100% Completed | 15.1s


In [19]:
import gdal

In [20]:
vrt = gdal.BuildVRT(os.path.join(data_path, 'TEAK_covariance_features.vrt'), stacks)
mosaic_sMonica = gdal.Translate(os.path.join(data_path, 'TEAK_covariance_features.tif'), vrt)

## old hyper stax ##

In [14]:
def make_hyper_lidar_tif(f, data_path, extent, t0, verbose=False, vverbose=False):
    '''TODO: make it verify that the same srs is in use fo extent and bbox'''
    
    # Contemplate the verbosity
    if vverbose: verbose = True
        
    # make sure url is still valid
    refresh_url(f, t0)
    
    # name of file to be stored
    name = os.path.join(data_path, f['name'])
    size = f['size']
    
    # Download the laz
    download_from_NEON_API(f, data_path)
    if verbose: print('Download completed.')
   
    # find the bounds
    try:
        cmd = f'pdal info {name}'
        if verbose: print(f'About to call {cmd}')
        reply = subprocess.run(cmd, shell=True, capture_output=True)
        if vverbose:print(f'stdout was:\n\n{reply.stdout}\n---------------------------')
        meta = json.loads(reply.stdout)
        if vverbose: print(f'The json looks like:\n\n {meta}\n---------------------------')
        bbox = meta['stats']['bbox']['native']['bbox']
        bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])
        if verbose: print(f'Bounds are:\n{bounds}\n')
    
        # if the bounds are at least partially within the extent...
        a = bbox['minx'] <= extent['maxx']
        b = bbox['maxx'] >= extent['minx']
        c = bbox['miny'] <= extent['maxy']
        d = bbox['maxy'] >= extent['miny']
        if a and b and c and d:
            # make and execute the pdal pipeline
            pipeline, tile = make_pipe(name, bounds, data_path, resolution=1)
            count = pipeline.execute()

            # remove the laz file
            os.remove(name)

            # get and sort the layers to stack
            layers = [item for item in os.listdir(data_path) if tile in item]
            layers.sort()
            if vverbose: print(f'Layers are:\n{layers}')

            # make tags for the bands    
            tags = [l.rpartition('_')[2].split('.')[0] for l in layers]
            if verbose: print(f'Tags are:\m{tags}')

            # Read metadata of first layer
            lyr = os.path.join(data_path, layers[0])
            with rasterio.open(lyr) as src0:
                meta = src0.meta

            # Update meta to reflect the number of layers
            meta.update(count = len(layers))

            # Read each layer and write it to stack
            dst_file = os.path.join(data_path, f'lidar_stack_{tile}.tif')
            with rasterio.open(dst_file, 'w', **meta) as dst:
                for id, layer in enumerate(layers, start=1):
                    lyr = os.path.join(data_path, layer)
                    with rasterio.open(lyr) as src1:
                        dst.write_band(id, src1.read(1))

                for id, tag in enumerate(tags, start=1):
                    dst.update_tags(id, ColorInterp=tag)

            # delete the layer tifs
            for layer in layers:
                os.remove(os.path.join(data_path, layer))

        else:
            # remove the laz file
            os.remove(name)
    
    except:
        if os.path.isfile(name):
            os.remove(name)
            reason = 'processing'
        else:
            reason = 'download'
        
        # keep track of failed files and reason for failure
        return((f['name'], reason))
    

In [ ]:
t0, laz = generate_laz_download_info()

results = []
for f in laz:
    results.append(delayed(make_hyper_lidar_tif)(f, data_path, extent, t0))

with ProgressBar():
    computed = compute(*results)
    
computed = [f for f in computed if f != None]    
# computed contains failed file / reason tuples

[#                                       ] | 4% Completed | 35.1s

In [92]:
with open('pack.txt') as f:
    for i, line in enumerate(f):
        with open('requirements.txt', 'a') as dst:
            if i > 2:
                dst.write('=='.join(line.split()[:2]) + '\n')